In [1]:
import syft as sy
from syft.core.node.new.queue_stash import QueueItem

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/shubham/PySyft


In [2]:
domain = sy.Worker.named('Domain', processes=1)

SQLite Store Path:
!open file:///tmp/79fa33618d8eaa1fc55f6f8a2d34c65e.sqlite

> Starting Worker: Domain - 79fa33618d8eaa1fc55f6f8a2d34c65e - NodeType.DOMAIN - [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.metadata_service.MetadataService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>, <class 'syft.core.node.new.user_code_service.UserCodeService'>, <class 'syft.core.node.new.request_service.RequestService'>, <class 'syft.core.node.new.data_subject_service.DataSubjectService'>, <class 'syft.core.node.new.network_service.NetworkService'>, <class 'syft.core.node.new.policy_service.PolicyService'>, <class 'syft.core.node.new.message_service.MessageService'>, <class 'syft.core.node.new.project_service.ProjectService'>, <class 'syft.core.node.new.data_subject_member_service.DataSubjectMemberService'>]


In [3]:
client = domain.root_client

In [4]:
from syft.core.node.new.api import SyftAPICall

In [5]:
call = SyftAPICall(
    node_uid=domain.id, path="data_subject.get_all", args=(), blocking=False, kwargs={}
)

In [12]:
call.path

'data_subject.get_all'

In [6]:
signed_api_call = call.sign(client.credentials)

In [14]:
signed_api_call.message.path

'data_subject.get_all'

In [16]:
result = client.api.services.data_subject.get_all(blocking=False)

In [26]:
result

```python
class QueueItem:
  id: str = 4b613d4d84364bdfbfce2a3abfd3b471
  node_uid: str = 79fa33618d8eaa1fc55f6f8a2d34c65e
  result: str = syft.core.node.new.api.SignedSyftAPICall
  resolved: str = True

```

In [7]:
signed_result = domain.handle_api_call(signed_api_call)

In [10]:
signed_result.message.data

```python
class QueueItem:
  id: str = dcfdd77d4a25437094357f5810e4a5bd
  node_uid: str = 79fa33618d8eaa1fc55f6f8a2d34c65e
  result: str = None
  resolved: str = False

```

In [25]:
signed_result.message.data.resolve

SyftAPIData(data=[])

In [24]:
import zmq
from zmq import devices
import time

class MessageQueue:
    def __init__(self):
        self.sockets = []
        self.context = zmq.Context()
        
    def create_bound_pair(
        self, type1=zmq.PAIR, type2=zmq.PAIR, interface='tcp://127.0.0.1'
    ):
        """Create a bound socket pair using a random port."""
        s1 = self.context.socket(type1)
        s1.setsockopt(zmq.LINGER, 0)
        port = s1.bind_to_random_port(interface)
        s2 = self.context.socket(type2)
        s2.setsockopt(zmq.LINGER, 0)
        s2.connect(f'{interface}:{port}')
        self.sockets.extend([s1, s2])
        return s1, s2
    
    def build_queue(self):
        pub, sub = self.create_bound_pair(zmq.PUB, zmq.SUB)
        return pub, sub
        
    def build_device(self, mon_sub=b"", in_prefix=b'in', out_prefix=b'out'):
            self.device = devices.ThreadMonitoredQueue(zmq.PUB, zmq.SUB, zmq.PUB,
                                                in_prefix, out_prefix)
            alice = self.context.socket(zmq.PUB)
            bob = self.context.socket(zmq.SUB)
            mon = self.context.socket(zmq.SUB)

            aport = alice.bind_to_random_port('tcp://127.0.0.1')
            bport = bob.bind_to_random_port('tcp://127.0.0.1')
            mport = mon.bind_to_random_port('tcp://127.0.0.1')
            mon.setsockopt(zmq.SUBSCRIBE, mon_sub)

            self.device.connect_in("tcp://127.0.0.1:%i"%aport)
            self.device.connect_out("tcp://127.0.0.1:%i"%bport)
            self.device.connect_mon("tcp://127.0.0.1:%i"%mport)
            self.device.start()
            time.sleep(.2)
            try:
                # this is currenlty necessary to ensure no dropped monitor messages
                # see LIBZMQ-248 for more info
                mon.recv_multipart(zmq.NOBLOCK)
            except zmq.ZMQError:
                pass
            self.sockets.extend([alice, bob, mon])
            return alice, bob, mon
        
    def teardown_device(self):
        for socket in self.sockets:
            socket.close()
            del socket
        del self.device

In [40]:
import zmq
import random
import time


context = zmq.Context()
sockets = []


def ventilator(context):
    # Socket to send messages on
    sender = context.socket(zmq.PUSH)
    sender.bind("tcp://*:5557")
    sockets.append(sender)

    # Socket with direct access to the sink: used to synchronize start of batch
    sink = context.socket(zmq.PUSH)
    sink.connect("tcp://localhost:5558")
    sockets.append(sink)

    print("Press Enter when the workers are ready: ")
    _ = input()
    print("Sending tasks to workers...")

    # The first message is "0" and signals start of batch
    sink.send(b'0')

    # Initialize random number generator
    random.seed()

    # Send 100 tasks
    total_msec = 0
    for task_nbr in range(10):

        # Random workload from 1 to 100 msecs
        workload = task_nbr + 1 #random.randint(1, 100)
        total_msec += workload

        sender.send_string(f"{workload}")

    print(f"Total expected cost: {total_msec} msec")

    # Give 0MQ time to deliver
    time.sleep(1)

In [12]:
def worker(context):
    # Socket to receive messages on
    receiver = context.socket(zmq.PULL)
    receiver.connect("tcp://localhost:5557")
    sockets.append(receiver)

    # Socket to send messages to
    sender = context.socket(zmq.PUSH)
    sender.connect("tcp://localhost:5558")
    sockets.append(sender)

    # Process tasks forever
    while True:
        s = receiver.recv()

        # Simple progress indicator for the viewer
        sys.stdout.write('.')
        sys.stdout.flush()

        # Do the work
        time.sleep(int(s)*0.001)

        # Send results to sink
        sender.send(b'')

In [13]:
def sink(context):
    # Socket to receive messages on
    receiver = context.socket(zmq.PULL)
    receiver.bind("tcp://*:5558")
    sockets.append(receiver)

    # Wait for start of batch
    s = receiver.recv()

    # Start our clock now
    tstart = time.time()

    # Process 100 confirmations
    for task_nbr in range(100):
        s = receiver.recv()
        if task_nbr % 10 == 0:
            sys.stdout.write(':')
        else:
            sys.stdout.write('.')
        sys.stdout.flush()

    # Calculate and report duration of batch
    tend = time.time()
    print(f"Total elapsed time: {(tend-tstart)*1000} msec")

In [43]:
ventilator(context)

Press Enter when the workers are ready: 

Sending tasks to workers...
Total expected cost: 55 msec


In [42]:
for s in sockets:
    s.close()
sockets =[]